In [36]:
import json
import datetime

import pandas as pd
import numpy as np
from tqdm import tqdm

from h2ogpte import H2OGPTE

In [1]:
!ls data

jd_db.csv


In [4]:
df = pd.read_csv('data/jd_db.csv', encoding='sjis')
df

,業種,職種
0,自動車製造業,自動車設計エンジニア
1,自動車製造業,生産管理
2,自動車製造業,品質管理
3,自動車製造業,製造オペレーター
4,自動車製造業,購買担当
...,...,...
295,宇宙開発業,マーケティング担当
296,宇宙開発業,購買担当
297,宇宙開発業,物流担当
298,宇宙開発業,技術サポート


In [7]:
df.sample()

,業種,職種
158,エネルギー業,財務担当


In [10]:
with open('../_secret') as f:
    key = json.load(f)
H2OGPTE_KEY = key['test-collection-0726']

client = H2OGPTE(
    address='https://h2ogpte.internal.dedicated.h2o.ai/',
    api_key=H2OGPTE_KEY,
)

client

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.5.8`.
You can enable strict version checking by passing strict_version_check=True.


In [12]:
client.get_llm_names()

['mistralai/Mixtral-8x7B-Instruct-v0.1',
 'mistralai/Mistral-7B-Instruct-v0.3',
 'meta-llama/Meta-Llama-3.1-8B-Instruct',
 'meta-llama/Meta-Llama-3.1-70B-Instruct',
 'OpenGVLab/InternVL-Chat-V1-5',
 'h2oai/h2o-danube3-4b-chat',
 'mistral-medium-latest',
 'claude-3-5-sonnet-20240620',
 'claude-3-haiku-20240307',
 'gpt-35-turbo-1106',
 'gpt-4-1106-preview',
 'gpt-4o',
 'gemini-1.5-pro-latest']

In [27]:
chat_session_id = client.create_chat_session_on_default_collection()
chat_session_id

'b57b7694-4dc9-4763-88b6-efc3e7a90b84'

In [57]:
LlmModel = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

SysPrompt = '''
あなたは優秀なジョブディスクリプション作成AIです。（回答は必ず日本語で実施します。）
'''

job = df.sample()
job_industry = job['業種']
job_role = job['職種']
job_rank = np.random.choice(['ジュニア', 'ミドル', 'シニア'])

job = df.sample()
job_industry = job['業種'].values[0]
job_role = job['職種'].values[0]
job_rank = np.random.choice(['ジュニア', 'ミドル', 'シニア'])

Message = '''
以下の"募集情報"にの内容に従って、その職務に必要なスキルを箇条書きして下さい。
なお、出力は"注意事項"に従って下さい。

# 注意事項
- ランクですが、ジュニアだとスキル要件が簡単なもの、シニアだと高度なスキル要件が必要で、ミドルはその中間です。
- 出力は箇条描きのみです。箇条書きの記号は"-"を使います。
- 出力する項目数は5~10程度です。
- 出力は100~200字程度です。

# 募集情報
業種: {}
職種: {}
ランク: {}
'''.format(job_industry, job_role, job_rank)

print(Message)
print('---------------------------')

with client.connect(chat_session_id) as session:
    reply = session.query(
        message = Message,
        system_prompt=SysPrompt,
        llm=LlmModel,
        rag_config={"rag_type": "llm_only",},
    )
    print(reply.content)


以下の"募集情報"にの内容に従って、その職務に必要なスキルを箇条書きして下さい。
なお、出力は"注意事項"に従って下さい。

# 注意事項
- ランクですが、ジュニアだとスキル要件が簡単なもの、シニアだと高度なスキル要件が必要で、ミドルはその中間です。
- 出力は箇条描きのみです。箇条書きの記号は"-"を使います。
- 出力する項目数は5~10程度です。
- 出力は100~200字程度です。

# 募集情報
業種: 宇宙開発業
職種: ミッションプランナー
ランク: シニア

---------------------------
必要なスキル:

- 5年以上の宇宙開発業務 experience
- ミッション計画と実行の経験
- 宇宙船や衛星の設計・開発に携わる経験
- リスク管理と評価の経験
- 複雑なシステムの整理と調整能力
- 英語の流暢な会話能力
- チームでの協調と指導能力
- 政策や法規に基づいた計画立案能力
- 事業計画の策定と評価能力
- 安全性と信頼性の高いミッションプランの作成能力


In [63]:
LlmModel = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

SysPrompt = '''
あなたは優秀なジョブディスクリプション作成AIです。（回答は必ず日本語で実施します。）
'''

list_job_industry = []
list_job_role = []
list_job_rank = []
list_skill_sets = []

for i in tqdm(range(100)):
    job = df.sample()
    job_industry = job['業種']
    job_role = job['職種']
    job_rank = np.random.choice(['ジュニア', 'ミドル', 'シニア'])

    Message = '''
    以下の"募集情報"にの内容に従って、その職務に必要なスキルを箇条書きして下さい。
    なお、出力は"注意事項"に従って下さい。

    # 注意事項
    - ランクですが、ジュニアだとスキル要件が簡単なもの、シニアだと高度なスキル要件が必要で、ミドルはその中間です。
    - 出力は箇条描きのみです。箇条書きの記号は"-"を使います。
    - 出力する項目数は5~10程度です。
    - 出力は100~200字程度です。

    # 募集情報
    業種: {}
    職種: {}
    ランク: {}
    '''.format(job_industry, job_role, job_rank)

    with client.connect(chat_session_id) as session:
        reply = session.query(
            message = Message,
            system_prompt=SysPrompt,
            llm=LlmModel,
            rag_config={"rag_type": "llm_only",},
        )
    
    list_job_industry.append(job_industry)
    list_job_role.append(job_role)
    list_job_rank.append(job_rank)
    list_skill_sets.append(reply.content)


df_res =pd.DataFrame({
    'industry': list_job_industry,
    'role':list_job_role,
    'rank':list_job_rank,
    'skill_sets':list_skill_sets
})
now = datetime.datetime.now().strftime("%m-%d-%H_%M_%S")
fname = 'data/'+'jobdb_'+now+'.csv'
df_res.to_csv(fname)

100%|██████████| 100/100 [09:30<00:00,  5.71s/it]
